In [2]:
from crc import Calculator, Crc8
crcCalculator = Calculator(Crc8.CCITT)

In [30]:
fileZ = open("Z2.txt", "w")
fileW = open("W.txt", "r")
result = ""

# Czytanie ramek
lastByte = fileW.read(8)
while(lastByte != "01111110"):
    lastByte = lastByte[1:8] + fileW.read(1)
lastByte = fileW.read(8)
frame = ""
frames = []
while(len(lastByte) == 8):
    if(lastByte == "01111110"):
        if(frame != ""):
            frames.append(frame)
            frame = ""
        lastByte = fileW.read(8)
    else:
        frame += lastByte[0]
        lastByte = lastByte[1:8] + fileW.read(1)

# Odrozpychanie
unstuffedFrames = []
for frame in frames:
    unstuffedFrame = ""
    consecutiveOnes = 0
    for c in frame:
        if(consecutiveOnes < 5):
            unstuffedFrame += c
            if(c == "1"):
                consecutiveOnes += 1
            else:
                consecutiveOnes = 0
        elif(c == "1"):
            unstuffedFrame = ""
            break
        else:
            consecutiveOnes = 0
    if(unstuffedFrame != ""):
        unstuffedFrames.append(unstuffedFrame)

# Sprawdzenie crc 
for frame in unstuffedFrames:
    crc = int(frame[0:8], 2)
    msg = []
    l = 8
    r = 16
    while l < len(frame):
        msg.append(int(frame[l:min(r, len(frame))], 2))
        l += 8
        r += 8
    if(crcCalculator.checksum(bytes(msg)) == crc):
        fileZ.write(frame[8:len(frame)])

fileZ.close()
fileW.close()
